In [1]:
import os
from dotenv import load_dotenv
from langchain_neo4j import Neo4jGraph

load_dotenv()

neo4j_uri = os.getenv("NEO4J_URI")
neo4j_user = os.getenv("NEO4J_USERNAME")
neo4j_pass = os.getenv("NEO4J_PASSWORD")
graph = Neo4jGraph(neo4j_uri, neo4j_user, neo4j_pass, refresh_schema=False)

In [2]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_ollama import ChatOllama

llm = ChatOllama(model="gemma3:4b")
llm_transformer = LLMGraphTransformer(llm=llm)

In [3]:
from langchain_core.documents import Document

text = """
The backend team for the product Aone, managed by Product Manager Rishi, is organized with two lead developers: Sangeetha and Niveetha. All members of the backend team report up to Rishi. Sangeetha serves as the primary reporting lead for both senior and junior engineers. The senior engineers on the team are Vishali and Logesh, both reporting directly to Sangeetha. The junior engineers—Samuel, Siva Prakash, Ashok, and Vibin Kumar—also report to Sangeetha. Among them, Vibin Kumar is responsible for supervising the two interns: Sakthi Vel and Pooja Shree, who both report to him. This structure ensures a clear chain of responsibility from interns up through lead developers to the product manager.
"""
documents = [Document(page_content=text)]
graph_documents = await llm_transformer.aconvert_to_graph_documents(documents)

In [4]:
graph_documents[0].nodes

[Node(id='Rishi', type='Person', properties={}),
 Node(id='Sangeetha', type='Person', properties={}),
 Node(id='Niveetha', type='Person', properties={}),
 Node(id='Vishali', type='Person', properties={}),
 Node(id='Logesh', type='Person', properties={}),
 Node(id='Samuel', type='Person', properties={}),
 Node(id='Siva Prakash', type='Person', properties={}),
 Node(id='Ashok', type='Person', properties={}),
 Node(id='Vibin Kumar', type='Person', properties={}),
 Node(id='Sakthi Vel', type='Person', properties={}),
 Node(id='Pooja Shree', type='Person', properties={}),
 Node(id='Aone', type='Product', properties={})]

In [5]:
graph_documents[0].relationships

[Relationship(source=Node(id='Aone', type='Product', properties={}), target=Node(id='Rishi', type='Person', properties={}), type='MANAGED_BY', properties={}),
 Relationship(source=Node(id='Rishi', type='Person', properties={}), target=Node(id='Sangeetha', type='Person', properties={}), type='MANAGES', properties={}),
 Relationship(source=Node(id='Sangeetha', type='Person', properties={}), target=Node(id='Vishali', type='Person', properties={}), type='MANAGES', properties={}),
 Relationship(source=Node(id='Sangeetha', type='Person', properties={}), target=Node(id='Logesh', type='Person', properties={}), type='MANAGES', properties={}),
 Relationship(source=Node(id='Sangeetha', type='Person', properties={}), target=Node(id='Samuel', type='Person', properties={}), type='MANAGES', properties={}),
 Relationship(source=Node(id='Sangeetha', type='Person', properties={}), target=Node(id='Siva Prakash', type='Person', properties={}), type='MANAGES', properties={}),
 Relationship(source=Node(id='

In [6]:
graph.add_graph_documents(graph_documents, baseEntityLabel=True)